# Jobsheet 03 - Naive Bayes

**NIM**: 2041720132

**NAMA**: Maulana Bintang Irfansyah

**KELAS**: TI-3H

# Naive Bayes dengan Data Kategorikal

Pada percobaan kedua ini, kita akan menggunakan data riil untuk melakukan klasifikasi dengan Naive Bayes. Data yang digunakan adalah **salary.xlsx**. Data tersebut menggambarkan apakah seseorang akan memiliki gaji lebih dari $100K berdasarkan perusahaan tempat bekerja, jenis pekerjaan, dan tingkat pendidikan.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Inspeksi Data

Pada tahap ini kita akan melakukan loading data dan inspeksi data. Hal ini dilakukan untuk mengetahui apakah kita perlu melakukan proses pendahuluan sebelum melakukan training

In [ ]:
import numpy as np
import pandas as pd

# baca data
path = "/content/drive/MyDrive/Colab Notebooks/Machine Learning/Pertemuan 4/Melbourne_housing_FULL.csv"
data = pd.read_csv(path)

# Cek data teratas
data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,Abbotsford,68 Studley St,2,h,NaN,SS,Jellis,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,Yarra City Council,-37.8014,144.9958,Northern Metropolitan,4019.0
1,Abbotsford,85 Turner St,2,h,1480000.0,S,Biggin,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,Yarra City Council,-37.7996,144.9984,Northern Metropolitan,4019.0
2,Abbotsford,25 Bloomburg St,2,h,1035000.0,S,Biggin,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,Yarra City Council,-37.8079,144.9934,Northern Metropolitan,4019.0
3,Abbotsford,18/659 Victoria St,3,u,NaN,VB,Rounds,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,Yarra City Council,-37.8114,145.0116,Northern Metropolitan,4019.0
4,Abbotsford,5 Charles St,3,h,1465000.0,SP,Biggin,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,Yarra City Council,-37.8093,144.9944,Northern Metropolitan,4019.0


Berdasarkan pengecekan data, terdapat data bernilai kategorial pada fitur (variabel) **Suburb**, **Address**, ***Type***, ***Method***, **SellerG**, **CouncilArea**, dan ***Regionname***. Kita harus melakukan encoding terhadap nilai dari fitur tersebut. Meskipun secara sekilas nilai kategorial menunjukkan tendesi ke tipe data nominal, namun kita tidak akan menggunakan tendik encoding One Hot Encoder atau Ordinal Encoder. Naive Bayes berkerja berdasarkan prinsip probabilitas berkelompok (_join probability_). 

Hal ini membuat merepresentasikan nilai variabel dalam bentuk encoding sebetulnya tidak terlalu penting. Encoding diperlukan hanya untuk menamai ulang nilai kategori dalam bentuk angka dan kebutuhan library scikit-learn yang menggunakan angka sebagai parameter input.

Percobaan kali ini merupakan penerapan klasik algoritma Naive Bayes. Perhatikan kembali contoh intuisi perhitungan Naive Bayes pada modul Jobsheet 3. Meskipun nilai asli dari fitur tidak diketahui, kita masih dapat melakukan proses klasifikasi dengan Naive Bayes.

## Tahap Persiapan

Pada tahap ini kita akan melakukan beberapa hal, yaitu,

1. Encoding nilai kategorikal untuk kebutuhan training
2. Memisahkan fitur dan label
3. Split data training dan testing

### Encoding

In [ ]:
# Encoding
# Fungsi encoding yang akan digunakan adalah LabelEncoder
# Hal ini karena kita hanya mengganti nilai variabel dari nama berupa string menjadi angka. Sama halnya dengan label

from sklearn.preprocessing import LabelEncoder

# Inisiasi label encoder
encode = LabelEncoder()

# Terpakan label encoder
data['Suburb'] = encode.fit_transform(data['Suburb'])
data['Address'] = encode.fit_transform(data['Address'])
data['Type'] = encode.fit_transform(data['Type'])
data['Method'] = encode.fit_transform(data['Method'])
data['SellerG'] = encode.fit_transform(data['SellerG'])
data['CouncilArea'] = encode.fit_transform(data['CouncilArea'])
data['Regionname'] = encode.fit_transform(data['Regionname'])

# Cek hasil
data.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,0,29458,2,0,NaN,6,155,3/09/2016,2.5,3067.0,...,1.0,1.0,126.0,NaN,NaN,31,-37.8014,144.9958,2,4019.0
1,0,32512,2,0,1480000.0,2,33,3/12/2016,2.5,3067.0,...,1.0,1.0,202.0,NaN,NaN,31,-37.7996,144.9984,2,4019.0
2,0,15389,2,0,1035000.0,2,33,4/02/2016,2.5,3067.0,...,1.0,0.0,156.0,79.0,1900.0,31,-37.8079,144.9934,2,4019.0
3,0,9768,3,2,NaN,7,296,4/02/2016,2.5,3067.0,...,2.0,1.0,0.0,NaN,NaN,31,-37.8114,145.0116,2,4019.0
4,0,25128,3,0,1465000.0,5,33,4/03/2017,2.5,3067.0,...,2.0,0.0,134.0,150.0,1900.0,31,-37.8093,144.9944,2,4019.0


### Memisahkan Fitur dan Label

In [ ]:
X = data.iloc[:, :3:5]
y = data.iloc[:, 19]

### Split Data Training dan Testing

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

## Training dan Evaluasi Model

In [ ]:
# Kita akan menggunakan CategoricalNB untuk kasus ini
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score

# Inisasi CategoricalNB
cnb = CategoricalNB()

# Fit model
# Label y harus dalam bentu 1D atau (n_samples,)
cnb.fit(X_train, y_train)

# Prediksi dengan data training
y_train_pred = cnb.predict(X_train)

# Evaluasi akurasi training
acc_train = accuracy_score(y_train, y_train_pred)

# Prediksi test data
y_test_pred = cnb.predict(X_test)

# Evaluasi model dengan metric akurasi
acc_test = accuracy_score(y_test, y_test_pred)

# Print hasil evaluasi
print(f'Hasil akurasi data train: {acc_train}')
print(f'Hasil akurasi data test: {acc_test}')

Hasil akurasi data train: 0.9992827685135377
Hasil akurasi data test: 0.9992828456683879
